CREACION DEL MODELO DE RECOMENDACION

In [2]:
#Importamos las librerias a utilizar

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
#Leemos el archivo csv como un dataframe

df=pd.read_csv("Datasets/data_con_EDA.csv")

In [8]:
#Visualizamos el dataframe

df

,id,title,overview,popularity,release_year,genres,spoken_languages
0,862,Toy Story,led woody andys toy live happily room andys bi...,21.946943,1995,"['Animation', 'Comedy', 'Family']",['English']
1,8844,Jumanji,sibling judy peter discover enchanted board ga...,17.015539,1995,"['Adventure', 'Fantasy', 'Family']","['English', 'Français']"
2,15602,Grumpier Old Men,family wedding reignites ancient feud nextdoor...,11.712900,1995,"['Romance', 'Comedy']",['English']
3,31357,Waiting to Exhale,cheated mistreated stepped woman holding breat...,3.859495,1995,"['Comedy', 'Drama', 'Romance']",['English']
4,11862,Father of the Bride Part II,george bank recovered daughter wedding receive...,8.387519,1995,['Comedy'],['English']
...,...,...,...,...,...,...,...
37595,84419,House of Horrors,unsuccessful sculptor save madman named creepe...,0.222814,1946,"['Horror', 'Mystery', 'Thriller']",['English']
37596,390959,Shadow of the Blair Witch,truecrime documentary delve murder spree inspi...,0.076061,2000,"['Mystery', 'Horror']",['English']
37597,289923,The Burkittsville 7,film archivist revisits story rustin parr herm...,0.386450,2000,['Horror'],['English']
37598,222848,Caged Heat 3000,year 3000 ad world dangerous woman banished re...,0.661558,1995,['Science Fiction'],['English']


In [9]:
#Nos quedamos con las columnas que consideramos puedan generar un buen modelo de recomendacion 'genres' y "title"

columnas_deseadas = ['genres',"title"]
df = df[columnas_deseadas]

In [10]:
#Para facilitar la lectura de los datos exportamos ese dataframe con el que vamos a trabajr el modelo

df.to_csv("Datasets/data_modelo.csv", index=False)

In [3]:
#Leemos el archivo csv en forma de dataframe

df=pd.read_csv("Datasets/data_modelo.csv")

In [12]:
df

,genres,title
0,"['Animation', 'Comedy', 'Family']",Toy Story
1,"['Adventure', 'Fantasy', 'Family']",Jumanji
2,"['Romance', 'Comedy']",Grumpier Old Men
3,"['Comedy', 'Drama', 'Romance']",Waiting to Exhale
4,['Comedy'],Father of the Bride Part II
...,...,...
37595,"['Horror', 'Mystery', 'Thriller']",House of Horrors
37596,"['Mystery', 'Horror']",Shadow of the Blair Witch
37597,['Horror'],The Burkittsville 7
37598,['Science Fiction'],Caged Heat 3000


In [13]:
#Obtenemos la película de entrada
titulo="Avatar"
pelicula_entrada = df[df['title'] == titulo]

# Creamos una instancia de TfidfVectorizer para los géneros
tfidf_genres = TfidfVectorizer(tokenizer=lambda x: eval(x), lowercase=False)

# Creamos la matriz de características para los géneros
matriz_caracteristicas_genres = tfidf_genres.fit_transform(df['genres'].apply(str))

# Encontramos el índice de la película de entrada en el DataFrame actualizado
indice_genres = df[df['title'].str.contains(titulo, case=False)].index[0]

# Calculamos la similitud del coseno entre los géneros
similitud_genres = cosine_similarity(matriz_caracteristicas_genres[indice_genres], matriz_caracteristicas_genres)

# Obtenemos los índices de las películas más similares basadas en los géneros
indices_similares_genres = similitud_genres.argsort()[0][:][::-1]

# Obtenemos las películas recomendadas basadas en los géneros
peliculas_recomendadas_genres = df.loc[indices_similares_genres]

# Creamos una columna de similitud basada en los géneros
peliculas_recomendadas_genres['similitud_genres'] = similitud_genres[0, indices_similares_genres]

# Ordenamos el DataFrame por similitud basada en los géneros de manera descendente
df_recomendadas_genres = peliculas_recomendadas_genres.sort_values('similitud_genres', ascending=False)

# Reiniciamos los índices del DataFrame resultante
df_recomendadas_genres.reset_index(drop=True, inplace=True)

# Excluimos el valor de los géneros de la película de entrada del DataFrame
df_recomendadas_genres = df_recomendadas_genres[df_recomendadas_genres['title'] != titulo]

# Filtramos las películas con similitud de géneros mayor a cero
df_recomendadas_genres = df_recomendadas_genres[df_recomendadas_genres['similitud_genres'] > 0]

# Obtenemos las primeras películas recomendadas sin repeticiones
df_recomendadas_genres = df_recomendadas_genres.drop_duplicates(subset=['title'])

# Agregamos la fila correspondiente al título proporcionado al final del DataFrame
fila_titulo_proporcionado = df[df['title'] == titulo]
df_recomendadas_genres = pd.concat([df_recomendadas_genres, fila_titulo_proporcionado])

# Reiniciamos los índices del DataFrame resultante
df_recomendadas_genres.reset_index(drop=True, inplace=True)

#Visualizamos el dataframe
df_recomendadas_genres

c:\Users\Usuario\Desktop\Nueva carpeta\ambiente\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,genres,title,similitud_genres
0,"['Science Fiction', 'Action', 'Fantasy', 'Adve...",Threads of Destiny,1.000000
1,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Justice League,1.000000
2,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Dungeons & Dragons,1.000000
3,"['Science Fiction', 'Action', 'Adventure', 'Fa...",Godzilla vs. SpaceGodzilla,1.000000
4,"['Fantasy', 'Science Fiction', 'Action', 'Adve...",Mothra vs. Godzilla,1.000000
...,...,...,...
10074,"['Action', 'Comedy', 'Drama', 'Music', 'Romanc...","Stay Away, Joe",0.144747
10075,"['Drama', 'Action', 'Thriller', 'Mystery', 'Fa...",The Ten Lives of Titanics the Cat,0.143150
10076,"['Drama', 'Animation', 'War', 'Foreign', 'Acti...",Area 88,0.142663
10077,"['TV Movie', 'Action', 'Drama', 'History', 'War']",The Hunley,0.132962


In [14]:
# Creamos una matriz de características utilizando TF-IDF Vectorizer para 'title'
tfidf_title = TfidfVectorizer(stop_words='english')
matriz_caracteristicas_title = tfidf_title.fit_transform(df_recomendadas_genres['title'])

# Encontramos el índice de la película de entrada en el DataFrame actualizado
indice_title = len(df_recomendadas_genres) - 1

# Calculamos la similitud del coseno entre los títulos
similitud_title = cosine_similarity(matriz_caracteristicas_title[indice_title], matriz_caracteristicas_title)

# Obtenemos los índices de las películas más similares basadas en los títulos
indices_similares_title = similitud_title.argsort()[0][:][::-1]

# Obtenemos las películas recomendadas basadas en los títulos
peliculas_recomendadas_title = df_recomendadas_genres.loc[indices_similares_title]

# Creamos una columna de similitud basada en los títulos
peliculas_recomendadas_title['similitud_title'] = similitud_title[0, indices_similares_title]

# Ordenamos el DataFrame por similitud basada en los títulos de manera descendente
df_recomendadas_title = peliculas_recomendadas_title.sort_values('similitud_title', ascending=False)

# Reiniciamos los índices del DataFrame resultante
df_recomendadas_title.reset_index(drop=True, inplace=True)

#Visualizamos el dataframe
df_recomendadas_title.head(6)

,genres,title,similitud_genres,similitud_title
0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Avatar,NaN,1.0
1,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Avatar 2,1.000000,1.0
2,"['Documentary', 'Science Fiction']",My Avatar and Me,0.373705,1.0
3,"['Fantasy', 'Drama', 'Animation', 'War']",The Lily of Belgium,0.295673,0.0
4,"['Action', 'Crime', 'Drama', 'Family', 'Foreig...",Turning Point,0.147703,0.0
5,"['Action', 'Comedy', 'Music', 'Romance', 'West...",Annie Get Your Gun,0.148310,0.0


In [14]:
def recomendacion(titulo):

    #Si el titulo ingresado no esta en los titulos del dataset devolvemos el mensaje "No se tienen datos del titulo ingresado"
    if titulo not in df['title'].values:
        return {'mensaje': "No se tienen datos del titulo ingresado"}
    else:
    
        # Creamos una instancia de TfidfVectorizer para los géneros
        tfidf_genres = TfidfVectorizer(tokenizer=lambda x: eval(x), lowercase=False)

        # Creamos la matriz de características para los géneros
        matriz_caracteristicas_genres = tfidf_genres.fit_transform(df['genres'].apply(str))

        # Encontramos el índice de la película de entrada en el DataFrame actualizado
        indice_genres = df[df['title'].str.contains(titulo, case=False)].index[0]

        # Calculamos la similitud del coseno entre los géneros
        similitud_genres = cosine_similarity(matriz_caracteristicas_genres[indice_genres], matriz_caracteristicas_genres)

        # Obtenemos los índices de las películas más similares basadas en los géneros
        indices_similares_genres = similitud_genres.argsort()[0][:][::-1]

        # Obtenemos las películas recomendadas basadas en los géneros
        peliculas_recomendadas_genres = df.loc[indices_similares_genres]

        # Creamos una columna de similitud basada en los géneros
        peliculas_recomendadas_genres['similitud_genres'] = similitud_genres[0, indices_similares_genres]

        # Ordenamos el DataFrame por similitud basada en los géneros de manera descendente
        df_recomendadas_genres = peliculas_recomendadas_genres.sort_values('similitud_genres', ascending=False)

        # Reiniciamos los índices del DataFrame resultante
        df_recomendadas_genres.reset_index(drop=True, inplace=True)

        # Excluimos el valor de los géneros de la película de entrada del DataFrame
        df_recomendadas_genres = df_recomendadas_genres[df_recomendadas_genres['title'] != titulo]

        # Filtramos las películas con similitud de géneros mayor a cero
        df_recomendadas_genres = df_recomendadas_genres[df_recomendadas_genres['similitud_genres'] > 0]

        # Obtenemos las primeras películas recomendadas sin repeticiones
        df_recomendadas_genres = df_recomendadas_genres.drop_duplicates(subset=['title'])

        # Agregamos la fila correspondiente al título proporcionado al final del DataFrame
        fila_titulo_proporcionado = df[df['title'] == titulo]
        df_recomendadas_genres = pd.concat([df_recomendadas_genres, fila_titulo_proporcionado])

        # Reiniciamos los índices del DataFrame resultante
        df_recomendadas_genres.reset_index(drop=True, inplace=True)

        # Creamos una matriz de características utilizando TF-IDF Vectorizer para 'title'
        tfidf_title = TfidfVectorizer(stop_words='english')
        matriz_caracteristicas_title = tfidf_title.fit_transform(df_recomendadas_genres['title'])

        # Encontramos el índice de la película de entrada en el DataFrame actualizado
        indice_title = len(df_recomendadas_genres) - 1

        # Calculamos la similitud del coseno entre los títulos
        similitud_title = cosine_similarity(matriz_caracteristicas_title[indice_title], matriz_caracteristicas_title)

        # Obtenemos los índices de las películas más similares basadas en los títulos
        indices_similares_title = similitud_title.argsort()[0][:][::-1]

        # Obtenemos las películas recomendadas basadas en los títulos
        peliculas_recomendadas_title = df_recomendadas_genres.loc[indices_similares_title]

        # Creamos una columna de similitud basada en los títulos
        peliculas_recomendadas_title['similitud_title'] = similitud_title[0, indices_similares_title]

        # Ordenamos el DataFrame por similitud basada en los títulos de manera descendente
        df_recomendadas_title = peliculas_recomendadas_title.sort_values('similitud_title', ascending=False)

        # Reiniciamos los índices del DataFrame resultante
        df_recomendadas_title.reset_index(drop=True, inplace=True)

        # Excluimos el valor del título proporcionado del DataFrame
        df_recomendadas_title = df_recomendadas_title[df_recomendadas_title['title'] != titulo]

        # Obtenemos las primeras 5 películas recomendadas sin repeticiones
        df_recomendadas_title = df_recomendadas_title.drop_duplicates(subset=['title'])

        # Obtenemos una lista de los títulos recomendados
        lista_recomendados = df_recomendadas_title['title'].tolist()

        # Creamos el diccionario de salida
        recomendaciones = {'lista recomendada': lista_recomendados[:5]}

        #Devolvemos el diccionario
        return recomendaciones

In [15]:
recomendacion("hol")

{'mensaje': 'No se tienen datos del titulo ingresado'}